In [41]:
import multiprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold , cross_val_score
from sklearn.metrics import roc_auc_score
import xgboost as xgb

In [15]:
# reading the data
df_heart=pd.read_csv('../data/framingham.csv')
#df_heart.replace(np.nan,"NaN")
df_heart.dropna(axis=0,inplace=True)

##separation in X and y
X_heart = df_heart.drop( columns = "TenYearCHD" )
y_heart = df_heart[ "TenYearCHD" ]

In [34]:

from sklearn.model_selection import train_test_split

X , X_external, y, y_external = train_test_split(X_heart, y_heart, 
                                                      stratify=y_heart, 
                                                      random_state=94 , 
                                                      train_size=1000)
y.shape , y_external.shape

((1000,), (2658,))

In [17]:
from hyperopt import hp, fmin, tpe, rand, pyll, STATUS_OK, STATUS_FAIL, Trials, space_eval


space4xgb = {
    'n_estimators': pyll.scope.int(hp.quniform('n_estimators', 1 , 1000,1)),
    'eta': hp.loguniform('eta' , np.log(10**-4) , np.log(10**2) ),
    'max_depth': pyll.scope.int(hp.quniform('max_depth', 1, 16, 1)),
    'subsample': hp.uniform('subsample', 0.3, 1)
}

skf = StratifiedKFold(n_splits=5 , shuffle=True , random_state = 2024 )

## tentative solution
def hyperopt_xgb_train_CV5( params , X , y ):
    
    losses = -1 * cross_val_score( xgb.XGBClassifier(n_jobs=multiprocessing.cpu_count()-2 , **params) , 
                X , y, scoring = 'roc_auc' , cv = skf )
        
    return_dict = {'loss': np.mean(losses),
                   'loss_variance' : np.var(losses),
                   'status': STATUS_OK
                   }
    return return_dict

## simple cross-validation for tuning and evaluating

In [18]:
%%time
trials = Trials()

best = fmin(lambda params : hyperopt_xgb_train_CV5(params , X , y), 
            space4xgb, 
            algo=tpe.suggest, 
            max_evals=50, 
            trials=trials)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:34<00:00,  1.45trial/s, best loss: -0.715329253824072]
CPU times: user 3min 27s, sys: 1.66 s, total: 3min 29s
Wall time: 34.5 s


In [31]:
integer_params = ['max_depth' , 'n_estimators']
for p in integer_params:
    best[p] = int(best[p])
    
model = xgb.XGBClassifier(n_jobs=multiprocessing.cpu_count()-2 , **best)
model.fit(X,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.05383819823544669,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=1, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=145, n_jobs=6,
              num_parallel_tree=None, ...)

In [33]:
trials.best_trial['result']['loss']

-0.715329253824072

In [36]:
def simpleCV( X ,y ):
    trials = Trials()

    best = fmin(lambda params : hyperopt_xgb_train_CV5(params , X , y), 
                space4xgb, 
                algo=tpe.suggest, 
                max_evals=50, 
                trials=trials)
    integer_params = ['max_depth' , 'n_estimators']
    for p in integer_params:
        best[p] = int(best[p])

    model = xgb.XGBClassifier(n_jobs=multiprocessing.cpu_count()-2 , **best)
    model.fit(X,y)
    
    return model, -1 * trials.best_trial['result']['loss']


In [38]:
%%time
simpleCV( X.iloc[:100,:],y.iloc[:100] )

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:14<00:00,  3.45trial/s, best loss: -0.725796568627451]
CPU times: user 1min 28s, sys: 554 ms, total: 1min 29s
Wall time: 14.5 s


(XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eta=0.0004781321204878451,
               eval_metric=None, feature_types=None, gamma=None,
               grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=8, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=96, n_jobs=6,
               num_parallel_tree=None, ...),
 0.725796568627451)

## tuning with CV, evaluate with test set


In [42]:
def simpleCV_with_test( X ,y , test_fraction = 0.2):
    
    X_train , X_test, y_train, y_test = train_test_split(X, y, 
                                                      stratify=y, 
                                                      test_size=test_fraction)

    model , cv_score = simpleCV( X_train ,y_train )
    test_score = roc_auc_score( y_test , model.predict_proba( X_test )[:,1] )
    
    return model, test_score


In [43]:
%%time
simpleCV_with_test( X.iloc[:100,:],y.iloc[:100] )

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:16<00:00,  3.11trial/s, best loss: -0.569047619047619]
CPU times: user 1min 38s, sys: 683 ms, total: 1min 38s
Wall time: 16.2 s


(XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eta=0.000114938137267187,
               eval_metric=None, feature_types=None, gamma=None,
               grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=1, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=468, n_jobs=6,
               num_parallel_tree=None, ...),
 0.7254901960784313)

## nested CV


**exercise:** implement the nested CV youself:

In [ ]:
#scaffold:

def nestedCV( X ,y ):
    
    pass
    # ...
    
    return model, np.mean( test_scores )


In [78]:
#proposed solution
def nestedCV( X ,y ):
    
    skf_external = StratifiedKFold( n_splits=5 , shuffle=True )
    
    test_scores = []
    for t,v in skf_external.split(X,y):
        model , cv_score = simpleCV( X[ t,: ] ,y[t] )
        test_scores.append( roc_auc_score( y[v] , model.predict_proba( X[v,:] )[:,1] ) )

    model, cv_score = simpleCV( X ,y )
    
    return model, np.mean( test_scores )


In [74]:
%%time
nestedCV( X.iloc[:100,:],y.iloc[:100] )

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [62]:


from sklearn.datasets import make_classification

X_all, y_all = make_classification(
    n_samples=5000,
    n_features=30,
    n_informative=3,
    n_redundant=0,
    n_repeated=0,
    n_classes=2,
    n_clusters_per_class=1,
    class_sep=0.1,
    random_state=0,
)


In [63]:
X , X_external, y, y_external = train_test_split(X_all, y_all, 
                                                      stratify=y_all, 
                                                      random_state=94 , 
                                                      train_size=1000)

In [72]:
%%time 
model , score = simpleCV( X[:100,:],y[:100] )
score , roc_auc_score( y_external , model.predict_proba( X_external )[:,1] ) 

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:23<00:00,  2.12trial/s, best loss: -0.6675151515151516]
CPU times: user 2min 23s, sys: 1.17 s, total: 2min 24s
Wall time: 23.7 s


(0.6675151515151516, 0.6670240008040018)

In [66]:
%%time 
model , score = simpleCV_with_test( X[:100,:],y[:100] )
score , roc_auc_score( y_external , model.predict_proba( X_external )[:,1] ) 

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:14<00:00,  3.39trial/s, best loss: -0.6415426587301587]
CPU times: user 1min 31s, sys: 650 ms, total: 1min 31s
Wall time: 14.8 s


(0.5757575757575758, 0.5015160034110077)

In [ ]:
%%time 
model , score = nestedCV( X[:100,:],y[:100] )
score , roc_auc_score( y_external , model.predict_proba( X_external )[:,1] ) 

 40%|████████████████████████████████████████████████████████████████████████████████▊                                                                                                                         | 20/50 [00:09<00:12,  2.34trial/s, best loss: -0.6253939393939394]